# CO$_{2}$ Flux Decomposition

This is the working area to develop a script that takes time series from a given region and decomposes the CO2 flux changes from a given climate event (e.g. ENSO) into the relative components contributing to that change (U, SST, S, DIC, etc.)

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-whitegrid')
import seaborn as sns
from esmtools.physics import stress_to_speed
from esmtools.carbon import co2_sol, schmidt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import esmtools as et

In [2]:
"""
Some constants that are pertinent for this.
"""
a = 6.972e-7 # Flux coefficient in s/m

### California Current NPGO Test Case

In [ ]:
def load_timeseries(e, v):
    """
    Loads in the residuals time series for a given upwelling system (e)
    and variable (v). I think it makes more sense to compute senstivities
    for each individual run rather than the full ensemble.
    """
    filepath = ('/glade/p/work/rbrady/EBUS_BGC_Variability/' + v + '/' + e +
               '/filtered_output/' + e.lower() + '-' + v + '-residuals-AW-chavez-800km.nc')
    da = xr.open_dataarray(filepath)
    da.name = v
    return da

In [ ]:
variables = ['FG_CO2', 'SST', 'SALT', 'DIC', 'ALK', 'TAUX',
             'TAUY', 'TAUX2', 'TAUY2', 'sDIC', 'sALK']
ds = xr.Dataset()
for i, name in enumerate(variables):
    da = load_timeseries('CalCS', name)
    ds = xr.merge([ds, da])

In [ ]:
filepath = ('/glade/u/home/rbrady/work/EBUS_BGC_Variability/area_weighted_regional_regressions/CalCS/' +
            'FG_CO2/NPGO/NPGO.FG_CO2.CalCS.unsmoothed.area_weighted_regional_regression.lag0.nc')

In [ ]:
ds = xr.open_dataset(filepath)

In [ ]:
df = ds.to_dataframe()

In [58]:
filepath = '/glade/p/work/rbrady/EBUS_BGC_Variability/U/CalCS/U.001.CalCS.192001-201512.nc'

In [59]:
ds_u = xr.open_dataset(filepath)

In [60]:
test = ds_u.isel(nlat=12,nlon=4)

In [61]:
np.allclose(test.U,uu,equal_nan=True)

True

In [55]:
x = x['TAUX'].isel(nlat=12,nlon=4)
y = y['TAUY'].isel(nlat=12,nlon=4)

In [56]:
uu = et.physics.stress_to_speed(x,y)

In [57]:
uu

<xarray.DataArray (time: 1152)>
array([ 7.458993,  7.592981,  8.266263, ...,  5.673193,  6.658523,  6.313285])
Coordinates:
  * time     (time) datetime64[ns] 1920-01-31 1920-02-29 1920-03-31 ...

# TAU -> U

In [51]:
def open_ebus_variable(v, en, eb):
    """
    Opens the ensemble member dataset that has been extracted already.
    """
    filepath = ('/glade/p/work/rbrady/EBUS_BGC_Variability/' + v + '/' + eb + '/' + v + '.' + en +
               '.' + eb + '.192001-201512.nc')
    ds = xr.open_dataset(filepath)
    return ds

In [52]:
def sts(ds):
    x = ds.TAUX
    y = ds.TAUY
    if x.min().isnull(): # Check if on coastline
        U10 = np.zeros([x.time.size,])
        U10[:] = np.nan
    else:
        tau = (np.sqrt(x**2 + y**2)) / 1.2 * 100**2 / 1e5 # Convert from dyn/cm2 to m2/s2
        U10 = np.zeros(len(tau))
        for t in range(len(tau)):
            c_tau = tau[t]
            p = np.array([0.0000764, 0.000142, 0.0027, -1*c_tau])
            r = np.roots(p)
            i = np.imag(r)
            good = np.where(i == 0)
            U10[t] = np.real(r[good])
    return xr.DataArray(U10, dims=['time'], coords=[x['time']])

In [53]:
tau_type = 'TAU'
ens = '001'
ebu = 'CalCS'

In [54]:
x = open_ebus_variable('TAUX', ens, ebu)
y = open_ebus_variable('TAUY', ens, ebu)

In [ ]:
_ = np.zeros([x['nlat'].size, x.nlon.size, x.time.size])
_[:] = np.nan

In [ ]:
ds = xr.DataArray(_, dims=['nlat','nlon','time'], coords=[x.nlat, x.nlon, x.time])

In [ ]:
from esmtools.physics import stress_to_speed
for i in range(ds.nlat.size):
    for j in range(ds.nlon.size):
        xtemp = x['TAUX'].isel(nlat=i, nlon=j)
        ytemp = y['TAUY'].isel(nlat=i, nlon=j)
        if test.min().isnull():
            continue
        else:
            utemp = stress_to_speed(xtemp, ytemp)
            ds[i,j,:] = utemp
ds.name = 'U'